In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.decomposition import NMF, LatentDirichletAllocation
import string
from nltk.stem import WordNetLemmatizer
from nltk import word_tokenize  

In [6]:
df = pd.read_excel("Petshop.xlsx")
#df = df.dropna()

In [7]:
df

,Unnamed: 0,title,descrição,local,salario,link,descrição completa
0,0,Banhista,"Realizar banhos em cachorros e gatos, atendim...",NaN,NaN,https://www.catho.com.br/vagas/banhista/202352...,"Realizar banhos em cachorros e gatos, atendime..."
1,1,Banhista e Tosador,"Organizar o ambiente de trabalho, por meio da...",NaN,NaN,https://www.catho.com.br/vagas/banhista-e-tosa...,"Organizar o ambiente de trabalho, por meio da ..."
2,2,Banhista E Tosador,"Se você é descomplicado, apaixonado por pet, ...",NaN,NaN,https://www.catho.com.br/vagas/banhista-e-tosa...,"Se você é descomplicado, apaixonado por pet, p..."
3,3,Banhista - Pet Shop,Atuará com banho e cuidados com higiene dos a...,NaN,NaN,https://www.catho.com.br/vagas/banhista-pet-sh...,Atuará com banho e cuidados com higiene dos an...
4,4,Banhistas - Pet Shop,"Responsável pelo banho e secagem dos animais,...",NaN,NaN,https://www.catho.com.br/vagas/banhistas-pet-s...,"Responsável pelo banho e secagem dos animais, ..."
...,...,...,...,...,...,...,...
88,88,Gerente de Loja,Gerenciamento de loja de departamentos do ram...,NaN,NaN,https://www.catho.com.br/vagas/gerente-de-loja...,Gerenciamento de loja de departamentos do ramo...
89,89,Vendedor,"Atuará na área de vendas, e sedempenhará as a...",NaN,NaN,https://www.catho.com.br/vagas/vendedor/201886...,"Atuará na área de vendas, e sedempenhará as at..."
90,90,Vendedor,"Atendimento ao cliente, venda de produtos par...",NaN,NaN,https://www.catho.com.br/vagas/vendedor/202308...,"Atendimento ao cliente, venda de produtos para..."
91,91,Representante Comercial,Irar dar continuidade no atendimento como pro...,NaN,NaN,https://www.catho.com.br/vagas/representante-c...,Irar dar continuidade no atendimento como pros...


In [6]:
#df = df[df['cargo'] == 'Analista de Controladoria']

In [8]:
skills = df['descrição completa']
len(skills)

93

In [9]:
skills.replace(['\n',
                 '^.*?Expect',
                 '^.*?Qualifications',
                 '^.*?Required',
                 '^.*?expected',
                 '^.*?Responsibilities',
                 '^.*?Requisitos',
                 '^.*?Requirements',
                 '^.*?Qualificações',
                 '^.*?QualificationsRequired1',
                 '^.*?você deve ter:',
                 '^.*?experiência',
                 '^.*?você:',
                 '^.*?Desejável',
                 '^.*?great',
                 '^.*?Looking For',
                 '^.*?ll Need',
                 '^.*?Conhecimento',
                 '^.*?se:',
                 '^.*?habilidades',
                 '^.*?se:',
                 '^.*?REQUISITOS'
                 ], '', regex=True)

bad_words = ['gender', 'experience', 'application', 'Apply', 'salary', 'todos', 'os', 'company', 'identity', 'sexual', 'orientation',
            'de', 'orientação', 'sexual', 'gênero', 'committed', 'toda', 'client', 'conhecimento',
            'world', 'year', 'save', 'São', 'Paulo', 'information', 'e', 'orientação', 'sexual', 'equal', 'oppotunity', 'ambiente', 'will',
            'Experiência', 'national origin', 'todas', 'work', 'de', 'da', 'years', 'pessoa', 'clients', 'Plano', 'creating',
            'employer', 'saúde', 'em', 'working', 'pessoas', 'mais', 'data', 'people', 'dia', 'one', 'knowledges', 'plataforma',
            'ou', 'benefício', 'para', 'software', 'opportunity', 'tecnologia', 'você', 'mais', 'solution', 'national', 'origin',
            'trabalhar', 'option', 'negócio', 'empresa', 'o', 'sicence', 'team', 'é', 'veteran', 'status', 'etc', 'raça', 'cor', 'belive',
            'nossa', 'uma', 'como', 'Scientist', 'ferramenta', 'projeto', 'que', 'job', 'benefícios', 'knowledge', 'toll', 's', 'modelo',
            'desconto', 'cultura', 'serviço', 'time', 'se', 'solutions', 'mercado', 'das', 'somos', 'problema', 'mundo', 'race', 'color',
            'vaga', 'pelo', 'ser', 'show', 'Seguro', 'Se', 'um', 'Um', 'tool', 'regard', 'without', 'make', 'ao', 'técnica', 'life',
            'interested', 'diversidade', 'proud', 'ability', 'sobre', 'options', 'using', 'área', 'nosso', 'na', 'seu', 'product', 'produto',
            'building', 'skill', 'model', 'religion', 'Share', 'receive', 'consideration', 'Aqui', 'vida', 'ferramentas', 'Vale', 'Refeição',
            'Strong', 'Pay', 'range', 'available', 'part', 'trabalho', 'Alimentação', 'employment', 'qualified', 'applicants', 'gympass',
            'está', 'comprometida', 'forma', 'Transporte', 'Yes', 'gente', 'melhor', 'lugar', 'believe', 'moment', 'próximo', 'deasafio',
            'dos', 'oportunidade', 'idade', 'new', 'Try', 'Premium', 'deficiência', 'sempre', 'criar', 'employee', 'problemas', 'unavailable',
            'Brasil', 'dado', 'hiring', 'trends', 'equipe', 'recent', 'temos', 'build', 'career', 'nós', 'diferencial', 'ma',
            'total', 'oferecemos', 'contato', 'tem', 'não', 'free', 'Full','of','to','is','avançado',
            'fazer','apuração','apurar','atuação','atuar','realização','realizar','elaoração','elaborar',
            'planejamento','planejar','apoio','apoiar','facilidade','revisão','revisar','execução','executar',
            'análise','analisar','validação','validar','participação','participar','confecção','confeccionar',
            'consolidação','consolidar','desenvolvimento','desenvolver','criação','criar','auxílio','avançado','intermediário'
            'auxiliar','acompanhamento','acompanhar','controle','controlar','revisão','revisar','básico','fluente',
            'formação','formação','formar','graduação','graduar','superior','cursar','cursando','graduação','graduar',
            'vivencia','vivência','atuar','atuação','experiencia','experiência','conhecimento','conhecer','habilidade',
            'capacidade','entendimento','entender','facilidade','avançado','intermediário','intermediario','basico']

In [10]:
#stop words
stopw = 'de a o que e do da em um para é com não uma os no se na por mais as dos como mas foi ao ele das tem à seu sua ou ser quando muito há nos já está eu também só pelo pela até isso ela entre era depois sem mesmo aos ter seus quem nas me esse eles estão você tinha foram essa num nem suas meu às minha têm numa pelos elas havia seja qual será nós tenho lhe deles essas esses pelas este fosse dele tu te vocês vos lhes meus minhas teu tua teus tuas nosso nossa nossos nossas dela delas esta estes estas aquele aquela aqueles aquelas isto aquilo estou está estamos estão estive esteve estivemos estiveram estava estávamos estavam estivera estivéramos esteja estejamos estejam estivesse estivéssemos estivessem estiver estivermos estiverem hei há havemos hão houve houvemos houveram houvera houvéramos haja hajamos hajam houvesse houvéssemos houvessem houver houvermos houverem houverei houverá houveremos houverão houveria houveríamos houveriam sou somos são era éramos eram fui foi fomos foram fora fôramos seja sejamos sejam fosse fôssemos fossem for formos forem serei será seremos serão seria seríamos seriam tenho tem temos tém tinha tínhamos tinham tive teve tivemos tiveram tivera tivéramos tenha tenhamos tenham tivesse tivéssemos tivessem tiver tivermos tiverem terei terá teremos terão teria teríamos teriam'.split()

stopw = [*stopw,*bad_words]

In [11]:
def tokenize(text):
    WNLTokens = []
    chars = []
    
    WNLemmatizer = WordNetLemmatizer()
    for ch in text:
        if ch not in string.punctuation:
            chars.append(ch)
        else:
            chars.append(' ')
    txt = ''.join(chars)        
        
    tokens = word_tokenize(txt)
    for t in tokens:
        WNLTokens.append(WNLemmatizer.lemmatize(t))
    
    return WNLTokens

In [12]:
tfidf = TfidfVectorizer(max_df=0.9,min_df=3,stop_words=stopw,tokenizer=tokenize,ngram_range=(1, 2))
dtmtfidf = tfidf.fit_transform(skills)


C:\Users\a92550\AppData\Roaming\Python\Python38\site-packages\sklearn\feature_extraction\text.py:396: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['alimentação', 'applicant', 'apply', 'aqui', 'brasil', 'es', 'forum', 'full', 'paulo', 'pay', 'plano', 'premium', 'refeição', 'scientist', 'seguro', 'share', 'strong', 'transporte', 'trend', 'try', 'vale', 'yes'] not in stop_words.
  warnings.warn(


In [13]:
%%time
LDA = LatentDirichletAllocation(n_components=1,random_state=42)
LDA.fit(dtmtfidf)

Wall time: 153 ms


LatentDirichletAllocation(n_components=1, random_state=42)

In [14]:
for i,topic in enumerate(LDA.components_):
    print(f'Top 20 for topic {i}')
    print([tfidf.get_feature_names()[c] for c in topic.argsort()[-30:]])
    print('\n')

Top 20 for topic 0
['máquina', 'procedimentos', 'responsável', 'vendas', 'veterinária', 'loja', 'tosador', 'limpeza', 'tosas', 'tesoura', 'organização', 'auxiliar', 'função', 'médio completo', 'cliente', 'serviços', 'completo', 'pet shop', 'shop', 'clientes', 'ensino médio', 'médio', 'produtos', 'banho tosa', 'ensino', 'pet', 'atendimento', 'banho', 'animais', 'tosa']




C:\Users\a92550\AppData\Roaming\Python\Python38\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
